# Logistic Regression
- Reminder
- Computing Hypothesis
- Computing Cost Function
- Evaluation
- Higher Implementation

## Reminder: Logistic **Regression**

### Hypothesis(pred)
$ H(X) = \frac{1}{1+e^{-W^T X}} $

### Cost
$ cost(W) = -\frac{1}{m} \sum y \log\left(H(x)\right) + (1-y) \left( \log(1-H(x) \right) $

 - If $y \simeq H(x)$, cost is near 0.
 - If $y \neq H(x)$, cost is high.

### Weight Update via Gradient Descent

$ W := W - \alpha \frac{\partial}{\partial W} cost(W) $

 - $\alpha$: Learning rate

### 데이터

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# For reproducibility
torch.manual_seed(1)

In [2]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]

In [3]:
x_train = torch.FloatTensor(x_data)
x_train

tensor([[1., 2.],
        [2., 3.],
        [3., 1.],
        [4., 3.],
        [5., 3.],
        [6., 2.]])

In [4]:
x_train.shape

torch.Size([6, 2])

In [5]:
y_train = torch.FloatTensor(y_data)
y_train

tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])

In [6]:
y_train.shape

torch.Size([6, 1])

### Computing the Hypothesis

Hypothesis(pred)

$ H(X) = \frac{1}{1+e^{-W^T X}} $

In [7]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


In [8]:
W = torch.zeros((x_train.shape[1], 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [9]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))
hypothesis

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)

In [10]:
print(hypothesis.shape)

torch.Size([6, 1])


Hypothesis(pred) Torch

$ H(X) = \frac{1}{1+e^{-W^T X}} $

In [11]:
print('1/(1+e^{-1}) equals: ', torch.sigmoid(torch.FloatTensor([1])))

1/(1+e^{-1}) equals:  tensor([0.7311])


In [12]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)

In [13]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
torch.Size([6, 1])


### Computing the Cost Function (Low-level)

$ cost(W) = -\frac{1}{m} \sum y \log\left(H(x)\right) + (1-y) \left( \log(1-H(x) \right) $

In [14]:
print(hypothesis)
print(y_train)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.]])


In [15]:
-(y_train * torch.log(hypothesis) + (1 - y_train) * (torch.log(1 - hypothesis)))

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)

In [16]:
losses = -(y_train * torch.log(hypothesis) + (1 - y_train) * (torch.log(1 - hypothesis)))
print(losses)

tensor([[0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931],
        [0.6931]], grad_fn=<NegBackward>)


In [17]:
cost = losses.mean()
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


### Training with Low-level Binary Cross Entropy Loss

In [18]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [19]:
# 모델 초기화
W = torch.zeros((x_train.shape[1], 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr = 1)

epochs = 1000
for epoch in range(epochs + 1):
    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)

    cost = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis)).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print('Epoch [{:4d}/{}] Cost: {:4f}'.format(
            epoch, epochs, cost.item()
        ))

Epoch [   0/1000] Cost: 0.693147
Epoch [ 100/1000] Cost: 0.134722
Epoch [ 200/1000] Cost: 0.080643
Epoch [ 300/1000] Cost: 0.057900
Epoch [ 400/1000] Cost: 0.045300
Epoch [ 500/1000] Cost: 0.037261
Epoch [ 600/1000] Cost: 0.031673
Epoch [ 700/1000] Cost: 0.027556
Epoch [ 800/1000] Cost: 0.024394
Epoch [ 900/1000] Cost: 0.021888
Epoch [1000/1000] Cost: 0.019852


In [20]:
hypothesis.squeeze().detach()

tensor([2.7711e-04, 3.1636e-02, 3.9014e-02, 9.5618e-01, 9.9823e-01, 9.9969e-01])

### Training with F.binary_cross_entropy

In [21]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]]
y_data = [[0], [0], [0], [1], [1], [1]]
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [22]:
# 모델 초기화
W = torch.zeros((x_train.shape[1], 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr = 1)

epochs = 100
for epoch in range(epochs + 1):
    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 10 == 0:
        pred = hypothesis >= torch.FloatTensor([0.5])
        correct_pred = pred.float() == y_train
        accuracy = correct_pred.sum().item() / len(correct_pred)
        print("Epoch [{:4d}/{}] Cost: {:.4f} Accuracy: {:2.2f}%".format(
            epoch, epochs, cost.item(), accuracy * 100
        ))

Epoch [   0/100] Cost: 0.6931 Accuracy: 50.00%
Epoch [  10/100] Cost: 0.6714 Accuracy: 66.67%
Epoch [  20/100] Cost: 0.5905 Accuracy: 83.33%
Epoch [  30/100] Cost: 0.4871 Accuracy: 83.33%
Epoch [  40/100] Cost: 0.4011 Accuracy: 83.33%
Epoch [  50/100] Cost: 0.3196 Accuracy: 83.33%
Epoch [  60/100] Cost: 0.2458 Accuracy: 83.33%
Epoch [  70/100] Cost: 0.1892 Accuracy: 100.00%
Epoch [  80/100] Cost: 0.1585 Accuracy: 100.00%
Epoch [  90/100] Cost: 0.1446 Accuracy: 100.00%
Epoch [ 100/100] Cost: 0.1347 Accuracy: 100.00%


## Loading Real Data

In [23]:
!git clone https://github.com/yeonwoo780/deeplearning_basic.git

fatal: destination path 'deeplearning_basic' already exists and is not an empty directory.


In [24]:
!ls -lia
!ls -lia deeplearning_basic
!ls -lia deeplearning_basic/Pytorch_dataset

total 20
4849688 drwxr-xr-x 1 root root 4096 Jun  4 02:47 .
4064287 drwxr-xr-x 1 root root 4096 Jun  4 01:51 ..
6553616 drwxr-xr-x 4 root root 4096 Jun  1 13:40 .config
4064295 drwxr-xr-x 8 root root 4096 Jun  4 02:47 deeplearning_basic
4849689 drwxr-xr-x 1 root root 4096 Jun  1 13:40 sample_data
total 44
4064295 drwxr-xr-x 8 root root 4096 Jun  4 02:47 .
4849688 drwxr-xr-x 1 root root 4096 Jun  4 02:47 ..
4064378 drwxr-xr-x 5 root root 4096 Jun  4 02:47 Chapter10~19
4064443 drwxr-xr-x 2 root root 4096 Jun  4 02:47 Chapter1~9
4064464 drwxr-xr-x 2 root root 4096 Jun  4 02:47 dataset
4064296 drwxr-xr-x 8 root root 4096 Jun  4 02:47 .git
4064343 -rw-r--r-- 1 root root   13 Jun  4 02:47 .gitignore
4064453 drwxr-xr-x 2 root root 4096 Jun  4 02:47 Pytorch
4064458 drwxr-xr-x 2 root root 4096 Jun  4 02:47 Pytorch_dataset
4064471 -rw-r--r-- 1 root root  392 Jun  4 02:47 pyvenv.cfg
4064463 -rw-r--r-- 1 root root 3326 Jun  4 02:47 README.md
total 112
4064458 drwxr-xr-x 2 root root  4096 Jun  4 02

In [25]:
!pwd

/content


In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os


xy = np.loadtxt('/content/deeplearning_basic/Pytorch_dataset/data-03-diabetes.csv',
                delimiter = ',', dtype = np.float32)

In [27]:
x_data = xy[:, :-1]
x_data

array([[-0.294118  ,  0.487437  ,  0.180328  , ...,  0.00149028,
        -0.53117   , -0.0333333 ],
       [-0.882353  , -0.145729  ,  0.0819672 , ..., -0.207153  ,
        -0.766866  , -0.666667  ],
       [-0.0588235 ,  0.839196  ,  0.0491803 , ..., -0.305514  ,
        -0.492741  , -0.633333  ],
       ...,
       [-0.411765  ,  0.21608   ,  0.180328  , ..., -0.219076  ,
        -0.857387  , -0.7       ],
       [-0.882353  ,  0.266332  , -0.0163934 , ..., -0.102832  ,
        -0.768574  , -0.133333  ],
       [-0.882353  , -0.0653266 ,  0.147541  , ..., -0.0938897 ,
        -0.797609  , -0.933333  ]], dtype=float32)

In [28]:
y_data = xy[:, -1]
y_data = y_data.reshape(-1,1)
y_data[:5]

array([[0.],
       [1.],
       [0.],
       [1.],
       [0.]], dtype=float32)

In [29]:
x_train = torch.FloatTensor(x_data)
x_train[:5]

tensor([[-0.2941,  0.4874,  0.1803, -0.2929,  0.0000,  0.0015, -0.5312, -0.0333],
        [-0.8824, -0.1457,  0.0820, -0.4141,  0.0000, -0.2072, -0.7669, -0.6667],
        [-0.0588,  0.8392,  0.0492,  0.0000,  0.0000, -0.3055, -0.4927, -0.6333],
        [-0.8824, -0.1055,  0.0820, -0.5354, -0.7778, -0.1624, -0.9240,  0.0000],
        [ 0.0000,  0.3769, -0.3443, -0.2929, -0.6028,  0.2846,  0.8873, -0.6000]])

In [30]:
x_train.shape

torch.Size([759, 8])

In [31]:
y_train = torch.FloatTensor(y_data)
y_train[:5]

tensor([[0.],
        [1.],
        [0.],
        [1.],
        [0.]])

### Training with Real Data using low-level Binary Cross Entropy Loss

In [32]:
# 모델 초기화
W = torch.zeros((8, 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr = 1)

epochs = 100
for epoch in range(epochs + 1):
    hypothesis = torch.sigmoid(x_train.matmul(W) + b)
    cost = -(y_train * torch.log(hypothesis) + (1 - y_train) * torch.log(1 - hypothesis)).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 10 == 0:
        pred = hypothesis >= torch.FloatTensor([0.5])
        correct_pred = pred.float() == y_train
        accuracy = correct_pred.sum().item() / len(correct_pred)
        print('Epoch [{:4d}/{}] Accuracy: {:2.2f}% Cost: {:.6f}'.format(
            epoch, epochs, accuracy, cost.item()
        ))

Epoch [   0/100] Accuracy: 0.65% Cost: 0.693147
Epoch [  10/100] Accuracy: 0.67% Cost: 0.572727
Epoch [  20/100] Accuracy: 0.72% Cost: 0.539493
Epoch [  30/100] Accuracy: 0.76% Cost: 0.519708
Epoch [  40/100] Accuracy: 0.77% Cost: 0.507066
Epoch [  50/100] Accuracy: 0.77% Cost: 0.498539
Epoch [  60/100] Accuracy: 0.77% Cost: 0.492549
Epoch [  70/100] Accuracy: 0.77% Cost: 0.488209
Epoch [  80/100] Accuracy: 0.76% Cost: 0.484985
Epoch [  90/100] Accuracy: 0.77% Cost: 0.482543
Epoch [ 100/100] Accuracy: 0.77% Cost: 0.480661


### Training with Real Data using F.binary_cross_entropy

In [33]:
# 모델 초기화
W = torch.zeros((x_train.shape[1], 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

epochs = 100
for epoch in range(epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 10번마다 로그 출력
    if epoch % 10 == 0:
        pred = hypothesis >= torch.FloatTensor([0.5])
        correct_pred = pred.float() == y_train
        accuracy = correct_pred.sum().item() / len(correct_pred)

        print('Epoch {:4d}/{} Accuracy: {:2.2f}% Cost: {:.6f}'.format(
            epoch, epochs, accuracy * 100, cost.item()
        ))

Epoch    0/100 Accuracy: 65.35% Cost: 0.693147
Epoch   10/100 Accuracy: 66.80% Cost: 0.572727
Epoch   20/100 Accuracy: 72.20% Cost: 0.539493
Epoch   30/100 Accuracy: 75.89% Cost: 0.519708
Epoch   40/100 Accuracy: 76.94% Cost: 0.507066
Epoch   50/100 Accuracy: 76.94% Cost: 0.498539
Epoch   60/100 Accuracy: 77.47% Cost: 0.492549
Epoch   70/100 Accuracy: 77.21% Cost: 0.488209
Epoch   80/100 Accuracy: 76.42% Cost: 0.484985
Epoch   90/100 Accuracy: 76.55% Cost: 0.482543
Epoch  100/100 Accuracy: 76.68% Cost: 0.480661


### Optional: High-level Implementation with nn.Module

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os

xy = np.loadtxt('/content/deeplearning_basic/Pytorch_dataset/data-03-diabetes.csv',
                delimiter = ',', dtype = np.float32)

x_data = xy[:, :-1]

y_data = xy[:, -1]
y_data = y_data.reshape(-1,1)

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

class BinaryClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(x_train.shape[1], 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [35]:
model = BinaryClassifier()

In [36]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1)

epochs = 100
for epoch in range(epochs + 1):
    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 10 == 0:
        pred = hypothesis >= torch.FloatTensor([0.5])
        correct_pred = pred.float() == y_train
        accuracy = correct_pred.sum().item() / len(correct_pred)
        print('Epoch {:4d}/{} Accuracy {:2.2f}% Cost: {:.6f}'.format(
            epoch, epochs, accuracy * 100, cost.item()
        ))

Epoch    0/100 Accuracy 45.72% Cost: 0.704829
Epoch   10/100 Accuracy 67.59% Cost: 0.572392
Epoch   20/100 Accuracy 73.25% Cost: 0.539563
Epoch   30/100 Accuracy 75.89% Cost: 0.520041
Epoch   40/100 Accuracy 76.15% Cost: 0.507561
Epoch   50/100 Accuracy 76.42% Cost: 0.499125
Epoch   60/100 Accuracy 77.21% Cost: 0.493177
Epoch   70/100 Accuracy 76.81% Cost: 0.488846
Epoch   80/100 Accuracy 76.28% Cost: 0.485612
Epoch   90/100 Accuracy 76.55% Cost: 0.483146
Epoch  100/100 Accuracy 76.81% Cost: 0.481234
